In [ ]:
##install and import necessary modules
##this code was originally designed and run in google colab
##use outside of colab may require modification
##if using colab, you may need to restart your runtime after installing modules,
##depending on enviornment at time of code running.
##due to potential module dependencies, we will install DeepTables later

!pip install scikit-learn==1.5.2
!pip install tensorflow==2.12.1
!pip install xgboost==2.0.2
!pip install shap
import shap
import sys
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
import seaborn as sn
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, f1_score, roc_curve, auc, precision_recall_curve, recall_score, confusion_matrix, brier_score_loss
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import resample
sn.set(style='whitegrid')

print("Python version:", sys.version)
print("scikit-learn version:", sklearn.__version__)
print("XGBoost version:", xgb.__version__)
print("shap version:", shap.__version__)

In [ ]:
##import your dataset
##mount google drive if using in colab. Replace <MOUNT_POINT> with the directory where you want to mount the drive (e.g., /content/drive).
drive.mount('<MOUNT_POINT>')

# Replace <YOUR_FILE_PATH> with the actual path inside your Google Drive (e.g., My Drive/FileNameHere).
file_path = '<MOUNT_POINT>/<YOUR_FILE_PATH>.csv'

In [ ]:
# Import data and specify missing values
data = pd.read_csv(file_path, na_values=['NA', 'N/A', 'NULL', ' ', '', '-99', '-98', '-99.0', '-99.00', '-98.0', '-98.00', 'NaN'])

# Filter out rows where 'TRAUMATYPE' is 26, 'Other/unspecified', or 'Burn'
try:
  exclude_values = ['26', 'Other/unspecified', 'Burn']
  data = data[~data['TRAUMATYPE'].isin(exclude_values)]
except:
  pass

In [ ]:
##check dataframe to ensure it appears as it should
data.head()

In [ ]:
##check for missing data
data.isnull().sum(axis=0)

In [ ]:
##create a dataframe of all complications/things not available on admission.  We can remove all of these from the X data set and pick one to be
#our Y dataset

complications_df=pd.DataFrame()
complications_list= [
                    'HC_CLABSI', 'HC_DEEPSSI', 'HC_DVTHROMBOSIS', 'HC_ALCOHOLWITHDRAWAL', 'HC_CARDARREST', 'HC_CAUTI',
                    'HC_EMBOLISM', 'HC_EXTREMITYCS', 'HC_INTUBATION', 'HC_KIDNEY', 'HC_MI', 'HC_ORGANSPACESSI',
                    'HC_OSTEOMYELITIS', 'HC_RESPIRATORY', 'HC_RETURNOR', 'HC_SEPSIS', 'HC_STROKECVA', 'HC_SUPERFICIALINCISIONSSI',
                    'HC_PRESSUREULCER', 'HC_UNPLANNEDICU', 'HC_VAPNEUMONIA',
                    ##'EDDISCHARGEDISPOSITION',
                    'HOSPDISCHARGEDISPOSITION',
                    ##'EDDISCHARGEHRS',
                    'WITHDRAWALLST',
                    'VTEPROPHYLAXISTYPE',
                    'TOTALICULOS',
                    'TOTALVENTDAYS',
                    'VTEPROPHYLAXISHRS',
                    'VTEPROPHYLAXISDAYS', 'MORTALITY', 'EDDISCHARGEDAYS','FINALDISCHARGEDAYS','FINALDISCHARGEHRS', 'HMRRHGCTRLSURGDAYS',  'WITHDRAWALLSTHRS',
                    ##'AMERICANINDIAN', 'ASIAN', 'BLACK', 'PACIFICISLANDER', 'RACEOTHER', 'WHITE', 'RACE_NA', 'RACE_UK',
                    'ISS_05'
                    , 'AIS_FACE', 'AIS_NECK', 'AIS_HEAD', 'AIS_THORAX', 'AIS_ABDOMEN', 'AIS_SPINE', 'AIS_UPPEREX', 'AIS_LOWEREX', 'AIS_SKIN', 'AIS_OTHER'
                    ##, 'VTEPPXStartOver48', 'VTEPPXStartOver24', 'ICUOver48', 'ICUOver24', 'VentOver48', 'VentOver24'
                    , 'VTEPPXStartOver72', 'VTEPPXStartOver96', 'ICUOver72', 'ICUOver96', 'VentOver72', 'VentOver96'
                    , 'FacilityTotalWLST', 'factilityTotalPatients', 'FacilityWLSTRate'
                    , 'facilityWLSTNew', 'WLSTRateNew', 'WLSTRateNewCensored'
                    ]
for c in complications_list:
    complications_df[c] = data[c]
complications_df

In [ ]:
##this is where we choose our outcome variable, in this case, WLST, and move it to a separate dataframe
Y_data = pd.DataFrame()
Y_data['WLST'] = data['WITHDRAWALLST']
Y_data

In [ ]:
##clean Y_data by replacing "Yes" and "No" vcalues with 0's and 1's

Y_data['WLST'] = Y_data['WLST'].replace({'Yes': 1, 'No': 0})
Y_data

In [ ]:
##remove all unwanted variables as defined above from the input space
X_data = data.drop(columns=complications_list)
X_data.shape

In [ ]:
##need to remove any cases with missing data for our outcome variable
Missing_Y = Y_data.isnull().sum(axis=0)
Missing_Y

In [ ]:
##here we find which rows in Y have missing values

bad_row_index_list=[]
for n in range(0, Y_data.shape[0]):
    n_missings=Y_data.iloc[n,:].isnull().sum()
    if n_missings>0:
        bad_row_index_list.append(n)
bad_row_index_list

In [ ]:
##now remove the bad rows in Y
Y_clean = Y_data.drop(bad_row_index_list, axis=0)
Y_clean

In [ ]:
##ensure all cases with missing values for the outcome have been dropped
Missing_Y_clean = Y_clean.isnull().sum(axis=0)
Missing_Y_clean

In [ ]:
##and remove bad rows in X
X_data=X_data.drop(bad_row_index_list, axis=0)

In [ ]:
##check which variables in the input space have missing variables
Missing = X_data.isnull().sum(axis=0)
Missing[Missing>0]

In [ ]:
##order variables with missing data by percentage
data_missing = (X_data.isnull().sum(axis=0)/X_data.shape[0]) * 100
data_missing

In [ ]:
##display variables withOUT mising data
data_missing[data_missing == 0].index

In [ ]:
#remove the good columns (no missing values) from data_missing
data_missing = data_missing.drop(data_missing[data_missing == 0].index)
data_missing

In [ ]:
#sort this in ascending order
data_missing = data_missing.sort_values(ascending=False)
data_missing

In [ ]:
##prepare to drop variables with >50% missing values
dropCutoff=50
bad_column_names = data_missing[data_missing >=dropCutoff].index
bad_column_names

In [ ]:
##actually drop bad variables
X_data_new=X_data.drop(columns=bad_column_names, axis=1)

##check for which variables still have missing data (<50% missing values)
Missing = X_data_new.isnull().sum(axis=0)
Missing[Missing>0]

In [ ]:
#check for columns with less than 50% missing that need to be cleaned
to_be_cleaned_column_names = data_missing[data_missing <50].index
to_be_cleaned_column_names

In [ ]:
##perform median imputation for continuous variable and mode imputation for categorical
for c in to_be_cleaned_column_names:
    v=X_data_new[c]#get values in this column
    v_valid=v[~v.isnull()] # get valid values
    if X_data_new[c].dtype == np.dtype('O'): # non-numeric values
        X_data_new[c]=X_data_new[c].fillna(v.value_counts().index[0]).astype(object) # the most frequent category
    else: # numeric
        X_data_new[c]=X_data_new[c].fillna(v_valid.median()) #replace nan with median value

In [ ]:
##confirm no more missing data in imput space
X_data_new.isnull().sum().sum()

In [ ]:
##verify cleaned dataframe appears as intended
X_data_new.head()

In [ ]:
# Rename the 'TRAUMATYPE' column to 'Penetrating' and map the values to 0 and 1
X_data_new['Penetrating'] = X_data_new['TRAUMATYPE'].map({'Penetrating': 1, 'Blunt': 0})

# Drop the old 'TRAUMATYPE' column
X_data_new.drop(columns=['TRAUMATYPE'], inplace=True)

In [ ]:
# Display the entire DataFrame without truncation
pd.set_option('display.max_columns', None)

# Get column names and data types
columns_info = []
for column_name, dtype in zip(X_data_new.columns, X_data_new.dtypes):
    columns_info.append(f"{column_name}: {dtype}")

formatted_columns_info = "\n".join(columns_info)

# Print column names and data types
print("Column Names and Data Types:")
print(formatted_columns_info)

In [ ]:
##remove any additional variables necessary
## Remove the "RACE" and "TRANSPORTMODE" columns, as these are composite varibles that have already been 1 hot encoded
columns_to_remove = ['RACE', 'TRANSPORTMODE']
X_data_new = X_data_new.drop(columns=columns_to_remove, errors='ignore')

In [ ]:
##first we will convert No's and Yes's to 0's and 1's to minimize the amount of double variables (want to avoid Yes/Nos being converted to 1-hot variables)
##want code to be reusable between different populations of input data.  Not every population will have all of these variables
##Therefore, will do everything within separate try/except blocks

try:
    X_data_new= X_data_new.replace({True: 1, 'Yes': 1, "Female": 1, False: 0, 'No': 0, "Male": 0})
except:
    pass
try:
    X_data_new['ETHNICITY'] = X_data_new['ETHNICITY'].replace({'Hispanic or Latino': 1, 'Not Hispanic or Latino': 0})
except:
    pass
try:
    X_data_new['EMSGCSEYE'] = X_data_new['EMSGCSEYE'].replace({'None': 1, 'To pressure': 2, 'To sound': 3,
                                                               'Spontaneous': 4})
except:
    pass
try:
    X_data_new['GCSEYE'] = X_data_new['GCSEYE'].replace({'None': 1, 'To pressure': 2, 'To sound': 3, 'Spontaneous': 4})
except:
    pass
try:
    X_data_new['EMSGCSVERBAL'] = X_data_new['EMSGCSVERBAL'].replace({'None': 1, 'Sounds': 2, 'Words': 3,
                                                                     'Confused': 4, 'Oriented': 5})
except:
    pass
try:
    X_data_new['EMSGCSMOTOR'] = X_data_new['EMSGCSMOTOR'].replace({'None': 1, 'Extension': 2, 'Abnormal Flexion': 3,
                                                                 'Normal Flexion': 4, 'Localising': 5, 'Obeys commands': 6})
except:
    pass
try:
    X_data_new['TBIGCSMOTOR'] = X_data_new['TBIGCSMOTOR'].replace({'None': 1, 'Extension': 2, 'Abnormal Flexion': 3,
                                                                 'Normal Flexion': 4, 'Localising': 5, 'Obeys commands': 6})
except:
    pass
try:
    X_data_new['GCSVERBAL'] = X_data_new['GCSVERBAL'].replace({'None': 1, 'Sounds': 2, 'Words': 3,
                                                               'Confused': 4, 'Orientated': 5})
except:
    pass
try:
    X_data_new['GCSMOTOR'] = X_data_new['GCSMOTOR'].replace({'None': 1, 'Extension': 2, 'Abnormal Flexion': 3,
                                                           'Normal Flexion': 4, 'Localising': 5, 'Obeys commands': 6})
except:
    pass
try:
    X_data_new['RESPIRATORYASSISTANCE'] = X_data_new['RESPIRATORYASSISTANCE'].replace({'Assisted Respiratory Rate': 1,
                                                                                   'Unassisted Respiratory Rate': 0})
except:
    pass
try:
    X_data_new['SUPPLEMENTALOXYGEN'] = X_data_new['SUPPLEMENTALOXYGEN'].replace({'Supplemental Oxygen': 1,
                                                                             'No Supplemental Oxygen': 0})
except:
    pass

X_data_new.head()

##male coded as 0
##female coded as 1

##not hispanic coded as 0
##hispanic coded as 1

In [ ]:
##need to convert categorical values to numerical values using one-hot encoding
categorical_column=[]
for c in X_data_new.columns:
    if X_data_new[c].dtype == np.dtype('O', 'category'): # non-numeric values
        categorical_column.append(c)
categorical_column

In [ ]:
##check how many variables we need to one-hot encode
len(categorical_column)

In [ ]:
##verify dataframe shape
X_data_new.shape

In [ ]:
##one-hot encode variables above
X_clean=pd.get_dummies(X_data_new, columns=categorical_column, sparse=False)
X_clean.shape

In [ ]:
##verify cleaned true label dataframe shape
Y_clean.shape

In [ ]:
##verify no missing data in the cleaned input space
X_clean.isnull().sum().sum()

In [ ]:
##drop patient ID's
X_clean.drop(['inc_key'], axis=1, inplace=True)

In [ ]:
##replace boolean values in binary variables to numeric values
X_clean = X_clean.replace({True: 1, False: 0})

In [ ]:
##verify dataframe appears as intended
X_clean.head()

In [ ]:
##split cleaned input space into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X_clean, Y_clean, test_size=0.2, random_state=0)

In [ ]:
##Before converting to Numpy arrays, we generate copies of thed data in tensor format to ensure we have access to
##tensor format data if needed

X_train_tensor=X_train.copy()
Y_train_tensor=Y_train.copy()
X_test_tensor=X_test.copy()
Y_test_tesnor=Y_test.copy()

In [ ]:
##convert sets to Numpy arrays:
X_train=X_train.values
Y_train=Y_train.values.reshape(-1)
X_test=X_test.values
Y_test=Y_test.values.reshape(-1)

In [ ]:
##now we have X_train, Y_train, X_test, Y_test as numpy arrays

scaler=StandardScaler()
#get the parameters of the transform
scaler.fit(X_train)
#normalize the features in the training set
X_train_s = scaler.transform(X_train)
#normalize the features in the test set
X_test_s = scaler.transform(X_test)

##lets also scale the tensor copies we created
X_train_tensor_s = scaler.transform(X_train_tensor)
X_test_tensor_s = scaler.transform(X_test_tensor)

In [ ]:
##further split the training set into a training and validation/calibration set
X_train_s_cal, X_val_s_cal, Y_train_cal, Y_val_cal = train_test_split(X_train_s, Y_train, test_size=0.2, random_state=0)

In [ ]:
##now, initialize XGBoost model using parameters determined from gridsearchCV hyperparameter optimization
model_gb=xgb.XGBClassifier(random_state=0)
model_best_gb = xgb.XGBClassifier(random_state=0, colsample_bytree=0.6, learning_rate=0.1, max_depth=7, n_estimators=150, subsample=0.8)

In [ ]:
# Get predicted probabilities for the positive class (WLST)
model_best_gb.fit(X_train_s_cal, Y_train_cal)
y_prob_gbo_mtp = model_best_gb.predict_proba(X_test_s)[:, 1]

# Compute AUROC
auroc_gbo = roc_auc_score(Y_test, y_prob_gbo_mtp)
print(f"AUROC on the test set: {auroc_gbo}")

In [ ]:
# Calibrate the model on the validation set
calibrated_model = CalibratedClassifierCV(estimator=model_best_gb, method='isotonic', cv='prefit')
calibrated_model.fit(X_val_s_cal, Y_val_cal)

In [ ]:
# 1. Get predicted probabilities for the positive class in calibrated model (WLST)
# ========================================
y_prob_gbo = calibrated_model.predict_proba(X_test_s)[:, 1]

# ========================================
# 2. Basic Metrics + Curves
# ========================================
# ------------------------- #
#       ROC Curve
# ------------------------- #
auroc_gbo = roc_auc_score(Y_test, y_prob_gbo)
print(f"AUROC on the test set: {auroc_gbo:.3f}")

fpr_gbo, tpr_gbo, thresholds_roc = roc_curve(Y_test, y_prob_gbo)
roc_auc_gbo = auc(fpr_gbo, tpr_gbo)

plt.figure(figsize=(8, 8))
plt.plot(fpr_gbo, tpr_gbo, color='darkorange', lw=2,
         label=f'ROC curve (AUC = {roc_auc_gbo:.3f})')
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

# ------------------------- #
#  Precision-Recall Curve
# ------------------------- #
precision, recall, thresholds_pr = precision_recall_curve(Y_test, y_prob_gbo)
pr_auc = auc(recall, precision)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='blue', label=f'PR curve (AUC={pr_auc:.3f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='best')
plt.show()

# ========================================
# 3. Confusion Matrix Stats @ threshold=0.5
# ========================================
custom_threshold = 0.5
y_pred_custom_threshold = (y_prob_gbo >= custom_threshold).astype(int)

conf_matrix = confusion_matrix(Y_test, y_pred_custom_threshold)
falseNeg, falsePos, trueNeg, truePos = conf_matrix[1, 0], conf_matrix[0, 1], conf_matrix[0, 0], conf_matrix[1, 1]
actualPos, actualNeg = truePos + falseNeg, trueNeg + falsePos
predPos, predNeg = truePos + falsePos, trueNeg + falseNeg

fpr = falsePos / actualNeg if actualNeg else 0
fnr = falseNeg / actualPos if actualPos else 0
tpr = truePos / actualPos if actualPos else 0
tnr = trueNeg / actualNeg if actualNeg else 0
accuracy = (truePos + trueNeg) / (actualPos + actualNeg) if (actualPos + actualNeg) else 0
ppV = truePos / predPos if predPos != 0 else 0
npV = trueNeg / predNeg if predNeg != 0 else 0

brier_score = brier_score_loss(Y_test, y_prob_gbo)

print("\n=== Threshold = 0.50 ===")
print("Confusion Matrix:")
print(conf_matrix)
print(f"False Positive Rate (FPR): {fpr:.3f}")
print(f"False Negative Rate (FNR): {fnr:.3f}")
print(f"True Positive Rate (TPR/Recall): {tpr:.3f}")
print(f"True Negative Rate (TNR/Specificity): {tnr:.3f}")
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision (PPV): {ppV:.3f}")
print(f"Negative Predictive Value (NPV): {npV:.3f}")
print(f"Brier Score: {brier_score:.4f}")

# ========================================
# 4. Find Threshold that Maximizes F1 (0.01 increments)
# ========================================
candidate_thresholds = np.linspace(0, 1, 101)  # from 0.00 to 1.00 in 0.01 steps
best_f1 = -1.0
best_f1_threshold = 0.0

for t in candidate_thresholds:
    y_pred = (y_prob_gbo >= t).astype(int)
    f1_val = f1_score(Y_test, y_pred)
    if f1_val > best_f1:
        best_f1 = f1_val
        best_f1_threshold = t

print(f"\nOptimal threshold for max F1 (nearest hundredth): {best_f1_threshold:.2f}")
print(f"Max F1 Score at that threshold: {best_f1:.3f}")

# ========================================
# 5. Find Threshold for >=90% Specificity (0.01 increments)
# ========================================
tnr_90_thresh = None

for t in candidate_thresholds:
    y_pred = (y_prob_gbo >= t).astype(int)
    cm = confusion_matrix(Y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    actual_neg = tn + fp
    if actual_neg == 0:
        continue

    specificity = tn / actual_neg  # TNR
    if specificity >= 0.90:
        tnr_90_thresh = t
        break  # first threshold that achieves TNR >= 0.90

if tnr_90_thresh is not None:
    print(f"\nThreshold achieving >=90% specificity (nearest hundredth): {tnr_90_thresh:.2f}")
    # Recompute confusion matrix & TPR at that threshold
    y_pred_90sp = (y_prob_gbo >= tnr_90_thresh).astype(int)
    cm_90 = confusion_matrix(Y_test, y_pred_90sp)
    tn_90, fp_90, fn_90, tp_90 = cm_90.ravel()
    tnr_val_90 = tn_90 / (tn_90 + fp_90) if (tn_90 + fp_90) else 0
    tpr_val_90 = tp_90 / (tp_90 + fn_90) if (tp_90 + fn_90) else 0
    print(f"Specificity (TNR) at {tnr_90_thresh:.2f}: {tnr_val_90:.3f}")
    print(f"Sensitivity (TPR) at {tnr_90_thresh:.2f}: {tpr_val_90:.3f}")
else:
    print("\nNo threshold found where TNR >= 0.90 in [0.00, 1.00].")

# ========================================
# 6. Decision Curve
# ========================================
def net_benefit(y_true, y_prob, thresholds):
    """
    NetBenefit = (TP/N) - (FP/N)*(threshold/(1-threshold))
    """
    N = len(y_true)
    NB = []
    for t in thresholds:
        y_pred = (y_prob >= t).astype(int)
        TP = np.sum((y_true == 1) & (y_pred == 1))
        FP = np.sum((y_true == 0) & (y_pred == 1))
        if t == 1.0:
            nb_t = 0
        else:
            nb_t = (TP / N) - (FP / N) * (t / (1 - t))
        NB.append(nb_t)
    return NB

decision_thresholds = np.linspace(0.0, 1.0, 101)
NB_model = net_benefit(Y_test, y_prob_gbo, decision_thresholds)

N = len(Y_test)
prevalence = np.mean(Y_test)  # fraction of actual positives
treat_all_nb = []
for t in decision_thresholds:
    if t == 1.0:
        treat_all_nb.append(0)
    else:
        treat_all_nb.append(prevalence - (1 - prevalence)*(t/(1-t)))

treat_none_nb = np.zeros_like(decision_thresholds)

plt.figure(figsize=(8, 6))
plt.plot(decision_thresholds, NB_model, label='Model', color='darkorange')
plt.plot(decision_thresholds, treat_all_nb, label='Treat All', color='red', linestyle='--')
plt.plot(decision_thresholds, treat_none_nb, label='Treat None', color='blue', linestyle=':')
plt.xlabel('Threshold Probability')
plt.ylabel('Net Benefit')
plt.title('Decision Curve Analysis')
plt.legend(loc='best')
plt.ylim([-0.3, 0.3])
plt.xlim([0, 1.0])
plt.show()

# ========================================
# 7. Define a Function to Compute Metrics
# ========================================
def compute_metrics(y_true, y_prob, threshold):
    """
    Computes performance metrics at a given threshold.

    Parameters:
    - y_true: True binary labels
    - y_prob: Predicted probabilities for the positive class
    - threshold: Threshold to convert probabilities to binary predictions

    Returns:
    - metrics_dict: Dictionary containing performance metrics
    """
    y_pred = (y_prob >= threshold).astype(int)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) else 0
    sensitivity = tp / (tp + fn) if (tp + fn) else 0  # TPR
    specificity = tn / (tn + fp) if (tn + fp) else 0  # TNR
    precision = tp / (tp + fp) if (tp + fp) else 0      # PPV
    npv = tn / (tn + fn) if (tn + fn) else 0            # NPV
    f1 = f1_score(y_true, y_pred) if (tp + fp + fn) else 0

    metrics_dict = {
        'Accuracy': accuracy,
        'Sensitivity (TPR)': sensitivity,
        'Specificity (TNR)': specificity,
        'Precision (PPV)': precision,
        'Negative Predictive Value (NPV)': npv,
        'F1 Score': f1
    }

    return metrics_dict

# ========================================
# 8. Create and Print Performance Tables
# ========================================
# Initialize an empty list to store metric dictionaries
performance_metrics = []

# Thresholds to report
thresholds_to_report = {
    'Max F1 Score Threshold': best_f1_threshold,
    '90% Specificity Threshold': tnr_90_thresh
}

# Compute metrics for each threshold
for desc, thresh in thresholds_to_report.items():
    if thresh is not None:
        metrics = compute_metrics(Y_test, y_prob_gbo, thresh)
        metrics['Threshold'] = f"{thresh:.2f}"
        performance_metrics.append(metrics)
    else:
        print(f"\n{desc} is not available as no threshold achieved >=90% specificity.")

# Create a DataFrame
df_performance = pd.DataFrame(performance_metrics)

# Reorder columns for better readability
df_performance = df_performance[['Threshold', 'Accuracy', 'Sensitivity (TPR)',
                                 'Specificity (TNR)', 'Precision (PPV)',
                                 'Negative Predictive Value (NPV)', 'F1 Score']]

# Set descriptive index
df_performance.index = thresholds_to_report.keys()

# Display the tables
print("\n=== Performance Metrics at Selected Thresholds ===")
print(df_performance.to_string(float_format="{:.3f}".format))

In [ ]:
##generate reliability diagram with 95% confidence interval to assess calibration

def bootstrap_calibration_curve(y_true, y_prob, n_bins=10, n_boot=1000, random_state=None):
    """
    1) Compute the original bin-based calibration curve.
    2) Bootstrap the dataset n_boot times, each time recalculating the bin-based
       fraction of positives (prob_true) and storing it.
    3) Return the original curve + 95% CI per bin (based on 2.5 and 97.5 percentiles).
    """
    # -----------------------------
    # Original calibration curve
    # -----------------------------
    # prob_true_orig, prob_pred_orig = calibration_curve(...) does binning internally.
    # But we want to fix n_bins and ensure consistent binning across bootstraps.
    # We'll do a manual binning approach here to keep consistent bin boundaries.

    # Define bin edges (equally spaced from 0 to 1)
    bin_edges = np.linspace(0, 1, n_bins + 1)
    # Digitize predicted probabilities
    bin_indices = np.digitize(y_prob, bin_edges) - 1
    bin_indices[bin_indices == n_bins] = n_bins - 1  # cap any == n_bins to last bin

    # Prepare arrays to hold the original bin stats
    prob_pred_orig = np.zeros(n_bins)
    prob_true_orig = np.zeros(n_bins)
    counts_in_bin = np.zeros(n_bins, dtype=int)

    # Fill in the stats for each bin
    for i in range(n_bins):
        mask = (bin_indices == i)
        counts_in_bin[i] = np.sum(mask)
        if counts_in_bin[i] > 0:
            prob_pred_orig[i] = np.mean(y_prob[mask])   # mean predicted prob in this bin
            prob_true_orig[i] = np.mean(y_true[mask])   # fraction of positives (actual)
        else:
            # If bin is empty, set to NaN
            prob_pred_orig[i] = np.nan
            prob_true_orig[i] = np.nan

    # Remove empty bins (NaN) from the original arrays
    valid_mask = ~np.isnan(prob_pred_orig)
    prob_pred_orig = prob_pred_orig[valid_mask]
    prob_true_orig = prob_true_orig[valid_mask]

    # -----------------------------
    # Bootstrap to get CIs
    # -----------------------------
    rng = np.random.RandomState(random_state) if random_state else np.random

    # We'll store the fraction of positives (prob_true) for each bin in each bootstrap
    # but only for the bins that were valid in the original data
    boot_prob_true = np.zeros((n_boot, sum(valid_mask)))

    n_data = len(y_true)
    data_idx = np.arange(n_data)

    for b in range(n_boot):
        # Sample with replacement
        sample_indices = rng.randint(0, n_data, size=n_data)
        y_true_b = y_true[sample_indices]
        y_prob_b = y_prob[sample_indices]

        # Repeat the binning steps
        bin_indices_b = np.digitize(y_prob_b, bin_edges) - 1
        bin_indices_b[bin_indices_b == n_bins] = n_bins - 1

        prob_true_b = np.zeros(n_bins)
        for i in range(n_bins):
            mask_b = (bin_indices_b == i)
            if np.sum(mask_b) > 0:
                prob_true_b[i] = np.mean(y_true_b[mask_b])
            else:
                prob_true_b[i] = np.nan

        # filter to only valid bins
        prob_true_b = prob_true_b[valid_mask]
        boot_prob_true[b, :] = prob_true_b

    # Compute 2.5th and 97.5th percentile per bin (column-wise)
    lower_ci = np.nanpercentile(boot_prob_true, 2.5, axis=0)
    upper_ci = np.nanpercentile(boot_prob_true, 97.5, axis=0)

    return prob_pred_orig, prob_true_orig, lower_ci, upper_ci

# -----------------------------------------------------------
# Now actually generate the figure
# -----------------------------------------------------------

# 1) Compute the calibration data + bootstrap CIs
prob_pred_orig, prob_true_orig, lower_ci, upper_ci = bootstrap_calibration_curve(
    Y_test, y_prob_gbo, n_bins=10, n_boot=1000, random_state=42
)

# 2) Brier score (overall calibration measure)
brier_score = brier_score_loss(Y_test, y_prob_gbo)
print(f"Brier Score: {brier_score:.4f}")

# 3) Plot
plt.figure(figsize=(8,6))
# Perfect calibration line
plt.plot([0,1],[0,1], 'k--', label='Perfect Calibration')

# Original calibration line
# Sort by prob_pred_orig so the line doesn't jump around
sort_idx = np.argsort(prob_pred_orig)
x_sorted = prob_pred_orig[sort_idx]
y_sorted = prob_true_orig[sort_idx]
lower_sorted = lower_ci[sort_idx]
upper_sorted = upper_ci[sort_idx]

plt.plot(x_sorted, y_sorted, marker='o', color='b', label='Calibration Curve')

# 4) Fill the 95% CI region
plt.fill_between(x_sorted, lower_sorted, upper_sorted, color='b', alpha=0.2,
                 label='95% CI')

plt.xlabel('Mean Predicted Probability')
plt.ylabel('Fraction of Positives')
plt.title('Reliability Diagram with 95% Confidence Interval (CI)')
plt.legend(loc='best')
plt.xlim([0,1])
plt.ylim([0,1])
plt.show()

In [ ]:
##now, comput 95% confidence interval based on 1000-sample bootstrapping

# Initialize your classifier (replace with your actual classifier)
classifier = model_best_gb

# Compute the AUROC on the original test set
original_auroc = auroc_gbo



# Number of bootstrap samples
n_bootstrap_samples = 1000

# Initialize an array to store bootstrapped AUROC values
bootstrap_aurocs = np.zeros(n_bootstrap_samples)

# Perform bootstrapping
for i in range(n_bootstrap_samples):
    # Generate a bootstrap sample
    bootstrap_indices = np.random.choice(len(Y_test_tesnor), len(Y_test_tesnor), replace=True)
    y_bootstrap = y_prob_gbo_mtp[bootstrap_indices]
    y_true_bootstrap = Y_test_tesnor.iloc[bootstrap_indices]

    # Compute AUROC on the bootstrap sample
    bootstrap_aurocs[i] = roc_auc_score(y_true_bootstrap, y_bootstrap)

# Calculate the confidence interval
confidence_interval = np.percentile(bootstrap_aurocs, [2.5, 97.5])

print("Original AUROC:", original_auroc)
print("95% Confidence Interval:", confidence_interval)

In [ ]:
##now, comput 95% confidence interval of the AUPRC based on 1000-sample bootstrapping


# Number of bootstrap iterations
n_iterations = 1000
# List to store bootstrapped AUPRC values
auprc_scores = []

# Original precision-recall calculation for actual data
precision, recall, thresholds = precision_recall_curve(Y_test, y_prob_gbo_mtp)
pr_auc = auc(recall, precision)

# Bootstrapping process to calculate 95% CI
for i in range(n_iterations):
    # Resample Y_test and y_prob_gbo_mtp with replacement
    Y_test_resampled, y_prob_resampled = resample(Y_test, y_prob_gbo_mtp)

    # Calculate precision and recall for the bootstrap sample
    precision_resampled, recall_resampled, _ = precision_recall_curve(Y_test_resampled, y_prob_resampled)

    # Calculate AUPRC for the resampled data
    pr_auc_resampled = auc(recall_resampled, precision_resampled)

    # Store the AUPRC score
    auprc_scores.append(pr_auc_resampled)

# Calculate the 95% confidence interval (2.5th and 97.5th percentiles)
lower_bound = np.percentile(auprc_scores, 2.5)
upper_bound = np.percentile(auprc_scores, 97.5)

# Print the original AUPRC and 95% CI
print(f'AUPRC: {pr_auc:.3f}')
print(f'95% CI for AUPRC: [{lower_bound:.3f}, {upper_bound:.3f}]')

In [ ]:
##now, use Shapley Additive Explanations for better assessment and visualization of feature imprtance

your_dataframe = X_train_tensor  # will use this to get column labels, so need the tensor
model=model_best_gb

# Calculate SHAP values for X_test
explainer = shap.TreeExplainer(model)
shap_values_test = explainer.shap_values(X_train_s_cal)

# Calculate mean absolute SHAP values
mean_abs_shap = np.abs(shap_values_test).mean(axis=0)

# Sort feature indices based on mean absolute SHAP values
sorted_indices = np.argsort(mean_abs_shap)

# Identify top 20 most important features
top_5_percent_indices = sorted_indices[-20:]

# Extract top 20 SHAP values and features
top_5_percent_shap_values = shap_values_test[:, top_5_percent_indices]
top_5_percent_feature_names = your_dataframe.columns[top_5_percent_indices]

# Create horizontal bar chart for top 20 most important features
fig1, ax1 = plt.subplots(figsize=(12, 6))
bars = ax1.barh(top_5_percent_feature_names, mean_abs_shap[top_5_percent_indices], color='lightblue')
ax1.set_xlabel('Mean Absolute SHAP Value')
ax1.set_title('Top 5% Most Important Features - Mean Absolute SHAP Values')
plt.tight_layout()
plt.show()

In [ ]:
##now to create a SHAP beeswarm plot
model=model_best_gb

# Create a SHAP explainer object
explainer = shap.Explainer(model)

# Calculate SHAP values for the test data
shap_values = explainer.shap_values(X_train_s_cal)

# Generate a summary plot
fig, ax = plt.subplots()
shap.summary_plot(shap_values, X_train_s_cal, feature_names=X_train_tensor.columns, max_display=20)

In [ ]:
##now, initialize penalized regression model with parameters based on GridsearchCV optimization and fit
model_best_lr=LogisticRegression(random_state=0, C=0.01, max_iter=300, penalty='l1', solver='liblinear')
model_best_lr.fit(X_train_s_cal, Y_train_cal)

In [ ]:
##test model
y_pred_prob_lro = model_best_lr.predict_proba(X_test_s)[:, 1]
auroc_lro = roc_auc_score(Y_test, y_pred_prob_lro)
print(f'AUROC: {auroc_lro}')

In [ ]:
##now, initialize random forest model with parameters based on GridsearchCV optimization and fit
model_best_rf=RandomForestClassifier(max_depth=30, max_features = 'sqrt', min_samples_split=5,
                                     min_samples_leaf=2, n_estimators=400, random_state=0)
model_best_rf.fit(X_train_s_cal, Y_train_cal)

In [ ]:
##test model
y_pred_prob_rfo = model_best_rf.predict_proba(X_test_s)[:, 1]
auroc_rfo = roc_auc_score(Y_test, y_pred_prob_rfo)
print(f'AUROC: {auroc_rfo}')

In [ ]:
##now, initialize KNN model with parameters based on GridsearchCV optimization and fit
model_best_knn=KNeighborsClassifier(n_neighbors=807)
model_best_knn.fit(X_train_s_cal, Y_train_cal)

In [ ]:
##test model
y_pred_prob_knno = model_best_knn.predict_proba(X_test_s)[:, 1]
auroc_knno = roc_auc_score(Y_test, y_pred_prob_knno)
print(f'AUROC: {auroc_knno}')

In [ ]:
##copy existing dataframes to use in neural networks

X_clean_nn_test=X_test_s.copy()
Y_clean_nn_test=Y_test.copy()

X_clean_nn_train=X_train_s_cal.copy()
Y_clean_nn_train=Y_train_cal.copy()

X_clean_nn_cal=X_val_s_cal.copy()
Y_clean_nn_cal=Y_val_cal.copy()

In [ ]:
##ensure data is in pandas dataframe
X_train_df = pd.DataFrame(X_clean_nn_train)
Y_train_s = pd.Series(Y_clean_nn_train)

X_val_df = pd.DataFrame(X_clean_nn_cal)
Y_val_s = pd.Series(Y_clean_nn_cal)

X_test_df = pd.DataFrame(X_clean_nn_test)
Y_test_s = pd.Series(Y_clean_nn_test)

In [ ]:
!pip install deeptables
##revert to sklearn 1.5 to resolve dependency issues
!pip install scikit-learn==1.5
import deeptables
print("dt version:", deeptables.__version__)
from deeptables.models.deeptable import DeepTable, ModelConfig
from deeptables.models.deepnets import DCN

In [ ]:
#initialize neural network model and fit
# `auto_discrete` is used to decide wether to discretize continous varibles automatically.
conf = ModelConfig(
    nets=DCN,
    metrics=['AUC', 'accuracy'],
    auto_discrete=True
)
dt = DeepTable(config=conf)
model, history = dt.fit( X_train_df, Y_train_s, epochs=100, validation_data=(X_val_df, Y_val_s))
score = dt.evaluate(X_test_df, Y_test_s)
preds = dt.predict(X_test_df)

In [ ]:
# Calculate ROC curve
y_pred_prob_ANN = dt.predict_proba(X_clean_nn_test)[:, 1]
fpr_ANN, tpr_ANN, thresholds = roc_curve(Y_clean_nn_test, y_pred_prob_ANN)

# Calculate the Area Under the ROC Curve (AUC)
roc_auc_ANN = auc(fpr_ANN, tpr_ANN)

# Plot ROC curve
plt.figure(figsize=(8, 8))
plt.plot(fpr_ANN, tpr_ANN, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc_ANN:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()